<a href="https://colab.research.google.com/github/dexter11235813/EVA_5/blob/main/assignment_3/assignment_3x.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sat Dec 12 18:24:00 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [27]:
# imports
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets
from torchvision.transforms import transforms
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from functools import partial
import torch.nn.functional as F

tqdm = partial(tqdm, leave=True, position=0)

In [92]:
class Net(nn.Module):
  
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(1, 10, kernel_size=3, bias=False, padding=1)
    self.conv2 = nn.Conv2d(10, 10, kernel_size=3, bias=False)
    self.conv3 = nn.Conv2d(10, 20, kernel_size=3, bias=False, padding=1)
    self.conv4 = nn.Conv2d(20, 20 , kernel_size=3, bias=False)
    self.conv5 = nn.Conv2d(20, 30, kernel_size=3, bias=False, padding=1)
    self.max_pool = nn.MaxPool2d(2,2)
    self.conv6 = nn.Conv2d(30, 62, kernel_size=3, bias=False)
    self.GAP = nn.AvgPool2d(3)
    self.relu = nn.ReLU()
    self.dropout = nn.Dropout(0.2)
    
  def forward(self, x):
    x = self.dropout(self.max_pool(self.relu(self.conv2(self.dropout(self.relu(self.conv1(x)))))))  # 28 x 28 x 1 -> 28 x 28 x 10 (RF = 3) -> 26 x 26 x 10 (RF = 5) -> 13 x 13 x 10 (RF = 10)
    x = self.dropout(self.max_pool(self.relu(self.conv4(self.dropout(self.relu(self.conv3(x)))))))  # 13 x 13 x 10 (RF = 10) -> 13 x 13 x 20 (RF = 12) -> 11 x 11 x 20 (RF = 14) -> 5 x 5 x 20 (RF = 28)
    x = self.dropout(self.relu(self.conv5(x))) # 5 x 5 x 20 (RF = 28) -> 5 x 5 x 30 (RF = 30)
    x = self.GAP(self.conv6(x)) # 5 x 5 x 30 (RF = 30) -> 3 x 3 x 62 (RF = 32)
    #return x
    #x = x.view(-1, 10)
    #print(x.shape)
    return F.log_softmax(x.view(-1, 62), dim=1) 

In [99]:
from torchsummary import summary
use_cuda = torch.cuda.is_available() 
device = torch.device("cuda" if use_cuda else "cpu") # moving model to CUDA, if available
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 28, 28]              90
              ReLU-2           [-1, 10, 28, 28]               0
           Dropout-3           [-1, 10, 28, 28]               0
            Conv2d-4           [-1, 10, 26, 26]             900
              ReLU-5           [-1, 10, 26, 26]               0
         MaxPool2d-6           [-1, 10, 13, 13]               0
           Dropout-7           [-1, 10, 13, 13]               0
            Conv2d-8           [-1, 20, 13, 13]           1,800
              ReLU-9           [-1, 20, 13, 13]               0
          Dropout-10           [-1, 20, 13, 13]               0
           Conv2d-11           [-1, 20, 11, 11]           3,600
             ReLU-12           [-1, 20, 11, 11]               0
        MaxPool2d-13             [-1, 20, 5, 5]               0
          Dropout-14             [-1, 2

In [100]:
# downloading Dataset

train_set = datasets.EMNIST(
    root="./data",
    train=True,
    download=True,
    split="byclass",
    transform=transforms.Compose(
        [    # Experimented with random vertical and horizontal flip transforms, the resulting model accuracy did not beat the baseline
            # transforms.RandomRotation(10),
            # transforms.RandomHorizontalFlip(),
            # transforms.RandomVerticalFlip(),
            transforms.ToTensor(),
        ]
    ),
)
test_set = datasets.EMNIST(
    root="./data",
    train=False,
    download=True,
    split="byclass",
    transform=transforms.Compose(
        [
            # transforms.RandomRotation(10),
            # transforms.RandomHorizontalFlip(),
            # transforms.RandomVerticalFlip(),
            transforms.ToTensor(),
        ]
    ),
)

In [101]:
# Creating train_loader and test_loader from train and test set.
train_loader = torch.utils.data.DataLoader(train_set, batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=128, shuffle=True)

In [102]:
# batch = next(iter(train_loader))
# images, labels = batch
# grid = torchvision.utils.make_grid(images, nrow=8)
# plt.figure(figsize=(15, 15))
# plt.imshow(np.transpose(grid, (1,2,0)))
# print(labels)


In [103]:

def train(model, device, train_loader, optimizer, epoch):
    model.train() # setting the model on train
    train_loss = 0
    train_correct = 0
    for _, (data, target) in tqdm(enumerate(train_loader), total=len(train_loader)):
        data, target = data.to(device), target.to(device) # reading in data and label batches from the train_loader
        optimizer.zero_grad() # flushing all the gradients before training 
        output = model(data) # passing the image data through the model
        loss = F.nll_loss(output.squeeze(), target) # calculating loss
        train_loss += loss.item()
        train_pred = output.argmax(dim=1, keepdim=True) # current label predictions from the model
        train_correct += train_pred.eq(target.view_as(train_pred)).sum().item() # accumulating all the correct predictions from the batch
        loss.backward() # backpropagating loss through the model
        optimizer.step() # updating the optimizer
        
    train_loss /= len(train_loader.dataset) # calculating the train loss for the epoch

    print(
      "\n Train set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n".format(
          train_loss,
          train_correct,
          len(train_loader.dataset),
          100.0 * train_correct / len(train_loader.dataset),
          
      )
  )




def test(model, device, test_loader):
    model.eval() # putting the model in eval mode 
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for _, (data, target) in tqdm(enumerate(test_loader), total=len(test_loader)):
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(
                output.squeeze(), target
            ).item()  # sum up batch loss
            pred = output.argmax(
                dim=1, keepdim=True
            )  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print(
        "\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n".format(
            test_loss,
            correct,
            len(test_loader.dataset),
            100.0 * correct / len(test_loader.dataset),
        )
    )



In [105]:
%%time
optimizer = torch.optim.Adam(model.parameters()) # initializing the adam optimizer

EPOCHS = 20
for epoch in range(EPOCHS):
    print(f'EPOCH {epoch + 1} / {EPOCHS}')
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 4/5453 [00:00<02:19, 38.93it/s]

EPOCH 1 / 20


  1%|          | 10/909 [00:00<00:09, 95.31it/s]


 Train set: Average loss: 0.0062, Accuracy: 523956/697932 (75%)



  0%|          | 5/5453 [00:00<01:50, 49.35it/s]

...


Test set: Average loss: 0.0030, Accuracy: 99999/116323 (86%)

EPOCH 20 / 20


  1%|          | 10/909 [00:00<00:09, 91.26it/s]


 Train set: Average loss: 0.0034, Accuracy: 589372/697932 (84%)



100%|██████████| 909/909 [00:09<00:00, 96.32it/s]


Test set: Average loss: 0.0031, Accuracy: 100270/116323 (86%)

CPU times: user 25min 16s, sys: 39.5 s, total: 25min 55s
Wall time: 25min 58s
